# Using FABRIC GPUs

Your compute nodes can include GPUs. These devices are made available as FABRIC components and can be added to your nodes like any other component.

This example notebook will demonstrate how to reserve and use Nvidia GPU devices on FABRIC.


## Setup the Experiment

#### Import FABRIC API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create a Node

The cells below help you create a slice that contains a single node with an attached GPU. 

### Select GPU Type and select the FABRIC Site

First decide on which GPU type you want to try - this will determine the subset of sites where your VM can be placed.

In [ ]:
# pick which GPU type we will use (execute this cell). 

# choices include
# GPU_RTX6000
# GPU_TeslaT4
# GPU_A30
# GPU_A40
GPU_CHOICE = 'GPU_A40' 

# don't edit - convert from GPU type to a resource column name
# to use in filter lambda function below
choice_to_column = {
    "GPU_RTX6000": "rtx6000_available",
    "GPU_TeslaT4": "tesla_t4_available",
    "GPU_A30": "a30_available",
    "GPU_A40": "a40_available"
}

column_name = choice_to_column.get(GPU_CHOICE, "Unknown")
print(f'{column_name=}')

Give the slice and the node in it meaningful names.

In [ ]:
# name the slice and the node 
slice_name=f'My Simple GPU Slice with {GPU_CHOICE}'
node_name='gpu-node'

print(f'Will create slice "{slice_name}" with node "{node_name}"')

Use a lambda filter to figure out which site the node will go to.

In [ ]:
# find a site with at least one available GPU of the selected type
site = fablib.get_random_site(filter_function=lambda x: x[column_name] > 0)
print(f'Preparing to create slice "{slice_name}" with node {node_name} in site {site}')

Create the desired slice with a GPU component. 

In [ ]:
# Create Slice. Note that by default submit() call will poll for 360 seconds every 10-20 seconds
# waiting for slice to come up. Normal expected time is around 2 minutes. 
slice = fablib.new_slice(name=slice_name)

# Add node with a 100G drive and a couple of CPU cores (default)
node = slice.add_node(name=node_name, site=site, disk=100)
node.add_component(model=GPU_CHOICE, name='gpu1')

#Submit Slice Request
slice.submit();

## Get the Slice

Retrieve the node information and save the management IP addresses.

In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.show();

## Get the Node

Retrieve the node information and save the management IP address.


In [ ]:
node = slice.get_node(node_name) 
node.show()

gpu = node.get_component('gpu1')
gpu.show();


## Deal with IPv6 sites

Most of FABRIC sites provide IPv6 management interfaces and there are a number of prominent resources on the internet, chiefly GitHub and NVidia website that are not present on IPv6. To deal with it we need to check the type of management IP address our GPU node received and if it is an IPv6 address, configure its DNS so it can reach IPv4 networks.

In [ ]:
from ipaddress import ip_address, IPv6Address    

isipv6_site = False
# If the node is an IPv6 Node then configure NAT64
if type(ip_address(node.get_management_ip())) is IPv6Address:
    isipv6_site = True
    print(f'Node {node.get_name()} has an IPv6 management address, will update DNS configuration')


In [ ]:
# this code will be executed if the node uses an IPv6 site. See the notebook 
# 'Access non-IPv6 services (i.e. GitHub) from IPv6 FABRIC nodes' for more details

if isipv6_site:
    node.upload_file('../accessing_ipv4_services_from_ipv6_nodes/nat64.sh', 'nat64.sh')
    stdout, stderr = node.execute(f'chmod +x nat64.sh && ./nat64.sh')
    print(f'Uploaded and executed NAT64 DNS setup script to node {node.get_name()}')
    

## Update the Node

Next we should update the Rocky (default) distribution with latest packages to ensure NVidia install process works as expected.

In [ ]:
command = "sudo dnf upgrade -q -y"
stdout, stderr = node.execute(command)

Don't forget to reboot so the update changes take effect

In [ ]:
reboot = 'sudo reboot'

print(reboot)
node.execute(reboot)

slice.wait_ssh(timeout=360,interval=10,progress=True)

print("Now testing SSH abilites to reconnect...",end="")
slice.update()
slice.test_ssh()
print("Reconnected!")

## GPU PCI Device

Run the command <code>lspci</code> to see your GPU PCI device(s). This is the raw GPU PCI device that is not yet configured for use.  You can use the GPUs as you would any GPUs.

View node1's GPU

In [ ]:
command = "sudo dnf install -q -y pciutils && lspci | grep 'NVIDIA\|3D controller'"

stdout, stderr = node.execute(command)

## Install Nvidia Drivers

Now, let's run the following commands to install the latest NVidia driver and the CUDA libraries and compiler.

In [ ]:
# source: https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html#rhel-8-rocky-8
commands = [
    'sudo dnf install -q -y https://dl.fedoraproject.org/pub/epel/epel-release-latest-8.noarch.rpm',
    'sudo dnf config-manager --add-repo https://developer.download.nvidia.com/compute/cuda/repos/rhel8/x86_64/cuda-rhel8.repo',
    'sudo dnf clean expire-cache',
    'sudo dnf module install -q -y nvidia-driver:latest-dkms',
    'sudo dnf install -q -y cuda'
]

print("Installing CUDA...")
for command in commands:
    stdout, stderr = node.execute(command)
print("Done installing CUDA. Now, reboot for the changes to take effect.")

And once CUDA is installed, reboot the machine.

In [ ]:
reboot = 'sudo reboot'

print(reboot)
node.execute(reboot)

slice.wait_ssh(timeout=360,interval=10,progress=True)

print("Now testing SSH abilites to reconnect...",end="")
slice.update()
slice.test_ssh()
print("Reconnected!")


## Testing the GPU and CUDA Installation

First, verify that the Nvidia drivers recognize the GPU by running `nvidia-smi`.

In [ ]:
stdout, stderr = node.execute("nvidia-smi")

print(f"stdout: {stdout}")

Now, let's upload the following "Hello World" CUDA program file to the node.

`hello-world.cu`

*Source: https://computer-graphics.se/multicore/pdf/hello-world.cu*

*Author: Ingemar Ragnemalm*

>This file is from *"The real "Hello World!" for CUDA, OpenCL and GLSL!"* (https://computer-graphics.se/hello-world-for-cuda.html), written by Ingemar Ragnemalm, programmer and CUDA teacher. The only changes (if you download the original file from the website) are to additionally `#include <unistd.h>`, as `sleep()` is now a fuction defined in the `unistd.h` library.

In [ ]:
node.upload_file('./hello-world.cu', 'hello-world.cu')

We now compile the `.cu` file using `nvcc`, the CUDA compiler tool installed with CUDA. In this example, we create an executable called `hello_world`.

In [ ]:
stdout, stderr = node.execute("/usr/local/cuda-12.1/bin/nvcc -o hello_world hello-world.cu")

Finally, run the executable:

In [ ]:
stdout, stderr = node.execute("./hello_world")

print(f"stdout: {stdout}")

If you see `Hello World!`, the CUDA program ran successfully. `World!` was computed on the GPU from an array of offsets being summed with the string `Hello `, and the resut was printed to stdout.

### Congratulations! You have now successfully run a program on a FABRIC GPU!

## Cleanup Your Experiment

In [ ]:
fablib.delete_slice(slice_name)